# Merge and Prepare Intermediate Datasets -- Medications and Patients

This notebook is the data engineering for creating the intermediate dataset for the "Medications" and "Patients" merged dataset. The script reads in the raw data provided by our project partner, drops records where FRDPersonnelStartDate is NULL, removes remaining duplicates, add tenure by month attribute, converts categorical data into numeric, and adds new numeric columns utilizing the the "One Hot Encoding" method, then outputs dataframe ro a CSV file. I will document and comment the notebook later. Enjoy!

In [ ]:
# Funtion to identify and print easy to understand variable types 
def get_var_category(series):
    unique_count = series.nunique(dropna=False)
    total_count = len(series)
    if pd.api.types.is_numeric_dtype(series):
        return 'Numerical'
    elif pd.api.types.is_datetime64_dtype(series):
        return 'Date'
    elif unique_count == total_count:
        return 'Text (Unique)'
    else:
        return 'Categorical'

def print_categories(df):
    for column_name in df.columns:
        print(column_name, ": ", get_var_category(df[column_name]))

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import os, time

# Setup HTML display
from IPython.core.display import display, HTML
# Notebook cell width adjustment
display(HTML('<style>.container { width:80% !important; }</style>'))

## Read in Medications dataset

In [3]:
# Read in Medications dataset and capture how long it takes
print(os.getcwd())
readStart=time.time()

dfMedications = pd.read_excel(r'./data/20210225-ems-raw-v04.xlsx', 
                        sheet_name='Medications',
                        na_values=['NA'])

# Stop the clock and calculate read time
readStop=time.time()
readTime=readStop-readStart
readMin=np.floor(readTime/60)
readSec=np.floor(readTime-(readMin*60))
print("The file was read in {0:.0f} minutes and {1:.0f} seconds.".format(readMin,readSec))

dfMedications.shape

C:\Users\MDL\iPython Scripts
The file was read in 1 minutes and 40 seconds.


(63168, 6)

## Read in Patients dataset

In [4]:
# Read in Patients dataset and capture how long it takes
print(os.getcwd())
readStart=time.time()

dfPatients = pd.read_excel(r'./data/20210225-ems-raw-v04.xlsx', 
                        sheet_name='Patients', 
                        na_values=['NA'])

# Stop the clock and calculate read time
readStop=time.time()
readTime=readStop-readStart
readMin=np.floor(readTime/60)
readSec=np.floor(readTime-(readMin*60))
print("The file was read in {0:.0f} minutes and {1:.0f} seconds.".format(readMin,readSec))

dfPatients.shape

C:\Users\MDL\iPython Scripts
The file was read in 1 minutes and 47 seconds.


(543774, 12)

# Working with Medications dataframe

In [5]:
# Capture Medications record count in variable
varMedicationsCount = len(dfMedications.index)
print('Medications record count: ', varMedicationsCount)

Medications record count:  63168


In [6]:
# Just getting a count of NULL values, but not dropping them yet
(" \nCount total NaN at each column in a Medications dataFrame : \n\n", dfMedications.isnull().sum())

(' \nCount total NaN at each column in a Medications dataFrame : \n\n',
 Dim_Medication_PK                    0
 PatientId                            0
 Medication_Given_RXCUI_Code          6
 Medication_Given_Description         6
 FRDPersonnelID                       0
 Medication_Administered_Date_Time    1
 dtype: int64)

In [7]:
print_categories(dfMedications)

Dim_Medication_PK :  Numerical
PatientId :  Numerical
Medication_Given_RXCUI_Code :  Numerical
Medication_Given_Description :  Categorical
FRDPersonnelID :  Categorical
Medication_Administered_Date_Time :  Date


In [8]:
# Drop Medication_Administered_Date_Time column which is not needed for analytics, 
# Dim_Medication_PK acts as a record-level unique identifier
dfMedications = dfMedications.drop(['Medication_Administered_Date_Time'],axis=1)
dfMedications.columns

Index(['Dim_Medication_PK', 'PatientId', 'Medication_Given_RXCUI_Code',
       'Medication_Given_Description', 'FRDPersonnelID'],
      dtype='object')

In [9]:
dfMedications.head()

,Dim_Medication_PK,PatientId,Medication_Given_RXCUI_Code,Medication_Given_Description,FRDPersonnelID
0,65740,479862,7806.0,Oxygen,1D18E8FC-EE92-E211-A596-78E7D18C3D20
1,65744,479876,26225.0,Ondansetron (Zofran),35CA45A3-9C25-4E4D-957F-2EA9E40B5974
2,65749,479916,125464.0,Normal saline,F73B4EF0-B70C-4960-9B92-8BF196A30B57
3,65750,479920,237648.0,Dextrose 10 % (D10),C387923A-F613-E411-A585-F305C4522FCB
4,65758,480011,125464.0,Normal saline,08CC5399-F492-E211-A596-78E7D18C3D20


In [10]:
dfMedications.loc[dfMedications['FRDPersonnelID'] == '9E5D40DC-5DC6-E411-80C7-001DD8B71D38']

,Dim_Medication_PK,PatientId,Medication_Given_RXCUI_Code,Medication_Given_Description,FRDPersonnelID
3860,84682,537112,26225.0,Ondansetron (Zofran),9E5D40DC-5DC6-E411-80C7-001DD8B71D38
3861,84682,537112,26225.0,Ondansetron (Zofran),9E5D40DC-5DC6-E411-80C7-001DD8B71D38
4877,119059,552215,4337.0,Fentanyl Citrate (Sublimaze),9E5D40DC-5DC6-E411-80C7-001DD8B71D38
4878,119059,552215,4337.0,Fentanyl Citrate (Sublimaze),9E5D40DC-5DC6-E411-80C7-001DD8B71D38
6732,127985,577130,6960.0,Midazolam (Versed),9E5D40DC-5DC6-E411-80C7-001DD8B71D38
...,...,...,...,...,...
62934,442175,1461295,1191.0,"Acetylsalicylic Acid (Aspirin, ASA)",9E5D40DC-5DC6-E411-80C7-001DD8B71D38
62935,442176,1461295,26225.0,Ondansetron (Zofran),9E5D40DC-5DC6-E411-80C7-001DD8B71D38
62936,442176,1461295,26225.0,Ondansetron (Zofran),9E5D40DC-5DC6-E411-80C7-001DD8B71D38
62965,442211,1461909,1191.0,"Acetylsalicylic Acid (Aspirin, ASA)",9E5D40DC-5DC6-E411-80C7-001DD8B71D38


In [11]:
# Drop duplicate Medications rows, these are related to where FRDPersonnelID = 9E5D40DC-5DC6-E411-80C7-001DD8B71D38
# 516 records, deduped expect to drop 258 records
dfMedications = pd.DataFrame.drop_duplicates(dfMedications)
dfMedications.shape

(62910, 5)

In [12]:
# Capture revised Medications record count in variable 
varMedicationsCount2 = len(dfMedications.index)
print('Medications revised record count: ', varMedicationsCount2)

Medications revised record count:  62910


In [13]:
# Display duplicate records dropped count
varMedicationsDiff = varMedicationsCount - varMedicationsCount2
print('Duplicate records count: ', varMedicationsDiff)

Duplicate records count:  258


# Working with Patients dataframe

In [14]:
# Capture Patients record count in variable 
varPatientsCount = len(dfPatients.index)
print('Patients record count: ', varPatientsCount)

Patients record count:  543774


In [15]:
print(" \nCount total NaN at each column in a Patients dataFrame : \n\n", dfPatients.isnull().sum())

 
Count total NaN at each column in a Patients dataFrame : 

 PatientId                     0
FRDPersonnelID                0
Shift                         0
UnitId                        0
FireStation                   0
Battalion                     0
PatientOutcome                2
PatientGender            131378
CrewMemberRoles               0
DispatchTime                  0
FRDPersonnelGender          474
FRDPersonnelStartDate       474
dtype: int64


Work on Patients dataset first

In [16]:
print_categories(dfPatients)

PatientId :  Numerical
FRDPersonnelID :  Categorical
Shift :  Categorical
UnitId :  Categorical
FireStation :  Numerical
Battalion :  Numerical
PatientOutcome :  Categorical
PatientGender :  Categorical
CrewMemberRoles :  Categorical
DispatchTime :  Date
FRDPersonnelGender :  Categorical
FRDPersonnelStartDate :  Date


## Keeping columns for Focus Questions 1 & 2
```
PatientId
FRDPersonnelID
PatientOutcome
PatientGender
DispatchTime
FRDPersonnelGender
FRDPersonnelStartDate
```

In [17]:
# Create new dataframe with only columns needed for Focus Question 1
dfPatientsSub = dfPatients.iloc[:, [0,1,6,7,9,10,11]]
dfPatientsSub.shape

(543774, 7)

In [18]:
dfPatientsSub.head()

,PatientId,FRDPersonnelID,PatientOutcome,PatientGender,DispatchTime,FRDPersonnelGender,FRDPersonnelStartDate
0,479838,6ED7C99E-9E01-E211-B5F5-78E7D18CFD3C,Standby (No Services Performed),NaN,2018-01-01 00:29:02,Male,1997-08-18
1,479838,F39A55F0-C619-E511-80CA-001DD8B71D38,Standby (No Services Performed),NaN,2018-01-01 00:29:02,Male,2007-02-19
2,479838,D6D3C99E-9E01-E211-B5F5-78E7D18CFD3C,Standby (No Services Performed),NaN,2018-01-01 00:29:02,Male,2008-08-18
3,479839,71D4C99E-9E01-E211-B5F5-78E7D18CFD3C,No Treatment/Transport Required,NaN,2018-01-01 00:40:54,Male,2011-10-24
4,479839,F9D4C99E-9E01-E211-B5F5-78E7D18CFD3C,No Treatment/Transport Required,NaN,2018-01-01 00:40:54,Male,2012-02-27


In [19]:
# Remove duplicate records 
dfPatientsSub = pd.DataFrame.drop_duplicates(dfPatientsSub)
dfPatientsSub.shape

(543718, 7)

In [20]:
# Capture revised Patients record count in variable
varPatientsSubCount = len(dfPatientsSub.index)
print('Patients record count: ', varPatientsSubCount)

Patients record count:  543718


In [21]:
# Display duplicate records dropped count
varPatientsSubDiff = varPatientsCount - varPatientsSubCount
print('Duplicate records count: ', varPatientsSubDiff)

Duplicate records count:  56


In [22]:
# Just getting a count of NULL values, but not dropping them yet
(" \nCount total NaN at each column in a Medications Sub dataFrame : \n\n", dfPatientsSub.isnull().sum())

(' \nCount total NaN at each column in a Medications Sub dataFrame : \n\n',
 PatientId                     0
 FRDPersonnelID                0
 PatientOutcome                2
 PatientGender            131365
 DispatchTime                  0
 FRDPersonnelGender          460
 FRDPersonnelStartDate       460
 dtype: int64)

In [23]:
# Drop NULL PatientGender records
# Don't run yet

#dfPatientsSub = dfPatientsSub.drop(dfPatientsSub[(dfPatientsSub.PatientGender.isnull())].index)
#dfPatientsSub.shape

In [24]:
# Drop NULL rows for Provider Gender and Start Date (FRDPersonnelGender and FRDPersonnelStartDate are both 460)
dfPatientsSub = dfPatientsSub.drop(dfPatientsSub[(dfPatientsSub.FRDPersonnelGender.isnull())].index)
dfPatientsSub.shape

(543258, 7)

In [25]:
# Capture revised Patients record count in variable 
varPatientsSubCount2 = len(dfPatientsSub.index)
print('Patients record count: ', varPatientsSubCount2)

Patients record count:  543258


In [26]:
# Display duplicate records dropped count
varPatientsSubDiff2 = varPatientsSubCount - varPatientsSubCount2
print('Duplicate records count: ', varPatientsSubDiff2)

Duplicate records count:  460


# Merge Medications and Patients datasets
### Start with an inner join

In [27]:
# Inner join on Patients and Medications example
dfMedPat = dfPatientsSub.merge(dfMedications, 
                        on=('PatientId','FRDPersonnelID'))
dfMedPat.shape

(58699, 10)

In [28]:
dfMedPat.head()

,PatientId,FRDPersonnelID,PatientOutcome,PatientGender,DispatchTime,FRDPersonnelGender,FRDPersonnelStartDate,Dim_Medication_PK,Medication_Given_RXCUI_Code,Medication_Given_Description
0,479862,1D18E8FC-EE92-E211-A596-78E7D18C3D20,Treated & Transported,Female,2018-01-01 00:44:31,Female,2012-09-24,65740,7806.0,Oxygen
1,479876,35CA45A3-9C25-4E4D-957F-2EA9E40B5974,Treated & Transported,Male,2018-01-01 00:42:53,Male,2016-02-08,65744,26225.0,Ondansetron (Zofran)
2,479916,F73B4EF0-B70C-4960-9B92-8BF196A30B57,Treated & Transported,Male,2018-01-01 01:48:31,Male,2016-02-08,65749,125464.0,Normal saline
3,480011,08CC5399-F492-E211-A596-78E7D18C3D20,Treated & Transported,Male,2018-01-01 03:02:53,Male,2012-09-24,65758,125464.0,Normal saline
4,480022,E10FD6B2-19A1-468A-825B-7EE0061DBB01,Treated & Transported,Female,2018-01-01 03:04:20,Male,2016-09-19,65759,26225.0,Ondansetron (Zofran)


In [29]:
(" \nCount total NaN at each column in a Medications+Patients dataFrame : \n\n", dfMedPat.isnull().sum())

(' \nCount total NaN at each column in a Medications+Patients dataFrame : \n\n',
 PatientId                        0
 FRDPersonnelID                   0
 PatientOutcome                   0
 PatientGender                   32
 DispatchTime                     0
 FRDPersonnelGender               0
 FRDPersonnelStartDate            0
 Dim_Medication_PK                0
 Medication_Given_RXCUI_Code      6
 Medication_Given_Description     6
 dtype: int64)

## Next perform a left outer join on Medications back to Medications+Patients where NULL

In [30]:
# Add Composite Key using PatientId and FRDPersonnelID to both Medications and Medications+Patients 
# datasets for left outer joins
dfMedications['PK'] = dfMedications['PatientId'].map(str) + '_' + dfMedications['FRDPersonnelID']
dfMedPat['_PK'] = dfMedPat['PatientId'].map(str) + '_' + dfMedPat['FRDPersonnelID']

In [31]:
dfMedPat.columns

Index(['PatientId', 'FRDPersonnelID', 'PatientOutcome', 'PatientGender',
       'DispatchTime', 'FRDPersonnelGender', 'FRDPersonnelStartDate',
       'Dim_Medication_PK', 'Medication_Given_RXCUI_Code',
       'Medication_Given_Description', '_PK'],
      dtype='object')

In [32]:
print(" \nCount total NaN at each column in a Medications+Patients dataFrame : \n\n", dfMedPat.isnull().sum())

 
Count total NaN at each column in a Medications+Patients dataFrame : 

 PatientId                        0
FRDPersonnelID                   0
PatientOutcome                   0
PatientGender                   32
DispatchTime                     0
FRDPersonnelGender               0
FRDPersonnelStartDate            0
Dim_Medication_PK                0
Medication_Given_RXCUI_Code      6
Medication_Given_Description     6
_PK                              0
dtype: int64


In [33]:
# Perform left outer join on Composite Keys (PK and _PK) and create a new dataframe to append to exising dfMedPat
medPKi = dfMedications.set_index('PK')
medPatPKi = dfMedPat.set_index('_PK')

dfMedLeftJoin = dfMedications.merge(medPatPKi, 
                             how = 'left', 
                             left_on = ('PK'),
                             right_on = ('_PK'), 
                             indicator = 'i').query('i == "left_only"')

In [34]:
dfMedLeftJoin.shape

(4211, 17)

In [35]:
dfMedLeftJoin.head()

,Dim_Medication_PK_x,PatientId_x,Medication_Given_RXCUI_Code_x,Medication_Given_Description_x,FRDPersonnelID_x,PK,PatientId_y,FRDPersonnelID_y,PatientOutcome,PatientGender,DispatchTime,FRDPersonnelGender,FRDPersonnelStartDate,Dim_Medication_PK_y,Medication_Given_RXCUI_Code_y,Medication_Given_Description_y,i
3,65750,479920,237648.0,Dextrose 10 % (D10),C387923A-F613-E411-A585-F305C4522FCB,479920_C387923A-F613-E411-A585-F305C4522FCB,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,left_only
8,65770,480042,4337.0,Fentanyl Citrate (Sublimaze),F1D2C99E-9E01-E211-B5F5-78E7D18CFD3C,480042_F1D2C99E-9E01-E211-B5F5-78E7D18CFD3C,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,left_only
70,65887,480388,26225.0,Ondansetron (Zofran),38D8C99E-9E01-E211-B5F5-78E7D18CFD3C,480388_38D8C99E-9E01-E211-B5F5-78E7D18CFD3C,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,left_only
123,65978,480637,1191.0,"Acetylsalicylic Acid (Aspirin, ASA)",1FD58791-CD19-E511-80CA-001DD8B71D38,480637_1FD58791-CD19-E511-80CA-001DD8B71D38,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,left_only
135,66016,480751,26225.0,Ondansetron (Zofran),38D8C99E-9E01-E211-B5F5-78E7D18CFD3C,480751_38D8C99E-9E01-E211-B5F5-78E7D18CFD3C,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,left_only


In [36]:
print(" \nCount total NaN at each column in a Medications+Patients dataFrame : \n\n", dfMedLeftJoin.isnull().sum())

 
Count total NaN at each column in a Medications+Patients dataFrame : 

 Dim_Medication_PK_x                  0
PatientId_x                          0
Medication_Given_RXCUI_Code_x        0
Medication_Given_Description_x       0
FRDPersonnelID_x                     0
PK                                   0
PatientId_y                       4211
FRDPersonnelID_y                  4211
PatientOutcome                    4211
PatientGender                     4211
DispatchTime                      4211
FRDPersonnelGender                4211
FRDPersonnelStartDate             4211
Dim_Medication_PK_y               4211
Medication_Given_RXCUI_Code_y     4211
Medication_Given_Description_y    4211
i                                    0
dtype: int64


In [37]:
# Create new dataframe without _x
# Rename columns with _x
dfMedLeftJoin.rename(columns = {'Dim_Medication_PK_x':'Dim_Medication_PK',
                                'PatientId_x':'PatientId',
                                'Medication_Given_RXCUI_Code_x':'Medication_Given_RXCUI_Code',
                                'Medication_Given_Description_x':'Medication_Given_Description',
                                'FRDPersonnelID_x':'FRDPersonnelID'}, inplace = True) 
dfMedLeftJoin.columns

Index(['Dim_Medication_PK', 'PatientId', 'Medication_Given_RXCUI_Code',
       'Medication_Given_Description', 'FRDPersonnelID', 'PK', 'PatientId_y',
       'FRDPersonnelID_y', 'PatientOutcome', 'PatientGender', 'DispatchTime',
       'FRDPersonnelGender', 'FRDPersonnelStartDate', 'Dim_Medication_PK_y',
       'Medication_Given_RXCUI_Code_y', 'Medication_Given_Description_y', 'i'],
      dtype='object')

In [38]:
# Drop all NaN, _y columns, don't need them anymore
dfMedLeftJoin = dfMedLeftJoin.drop(['PatientId_y', 'FRDPersonnelID_y', 'PatientOutcome', 'PatientGender',
                                    'DispatchTime', 'FRDPersonnelGender', 'FRDPersonnelStartDate',
                                    'Dim_Medication_PK_y', 'Medication_Given_RXCUI_Code_y', 
                                    'Medication_Given_Description_y', 'i'],axis=1)
dfMedLeftJoin.columns

Index(['Dim_Medication_PK', 'PatientId', 'Medication_Given_RXCUI_Code',
       'Medication_Given_Description', 'FRDPersonnelID', 'PK'],
      dtype='object')

In [39]:
dfMedLeftJoin.shape

(4211, 6)

In [40]:
print(" \nCount total NaN at each column in a Medications+Patients dataFrame : \n\n", dfMedLeftJoin.isnull().sum())

 
Count total NaN at each column in a Medications+Patients dataFrame : 

 Dim_Medication_PK               0
PatientId                       0
Medication_Given_RXCUI_Code     0
Medication_Given_Description    0
FRDPersonnelID                  0
PK                              0
dtype: int64


## Build lookup reference dataframe from Patients dataframe containing a distinct list of 
```
FRDPersonnelID
FRDPersonnelGender
FRDPersonnelStartDate
```
## Then join lookup dataframe luProvider with dfMedLeftJoin on PatientId only

In [41]:
# Create Lookup table for Providers
luProvider = dfPatients.groupby(['FRDPersonnelID','FRDPersonnelGender', 'FRDPersonnelStartDate']).size().reset_index(name='count')
luProvider.shape

(1830, 4)

In [42]:
luProvider.head(5)

,FRDPersonnelID,FRDPersonnelGender,FRDPersonnelStartDate,count
0,00D5C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,2008-04-28,1200
1,00D6C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,2007-09-17,388
2,00D7C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,1993-09-20,21
3,00D8C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,1991-06-03,5
4,00D9C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,2003-04-19,249


In [43]:
# Drop count column, not needed
luProvider = luProvider.drop(['count'],axis=1)

# Join lookup dataframe luProvider with dfMedLeftJoin on PatientId only

In [44]:
# Join lookup dataframe luProvider with dfProcLeftJoin on PatientId only
dfMedPatOrphans = dfMedLeftJoin.merge(luProvider, 
                        on=('FRDPersonnelID'))
dfMedPatOrphans.shape

(4208, 8)

In [45]:
dfMedPatOrphans.head()

,Dim_Medication_PK,PatientId,Medication_Given_RXCUI_Code,Medication_Given_Description,FRDPersonnelID,PK,FRDPersonnelGender,FRDPersonnelStartDate
0,65750,479920,237648.0,Dextrose 10 % (D10),C387923A-F613-E411-A585-F305C4522FCB,479920_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24
1,171166,607195,4337.0,Fentanyl Citrate (Sublimaze),C387923A-F613-E411-A585-F305C4522FCB,607195_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24
2,330909,625060,296.0,Adenosine (Adenocard),C387923A-F613-E411-A585-F305C4522FCB,625060_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24
3,330910,625060,296.0,Adenosine (Adenocard),C387923A-F613-E411-A585-F305C4522FCB,625060_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24
4,347351,674487,125464.0,Normal saline,C387923A-F613-E411-A585-F305C4522FCB,674487_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24


In [46]:
# The count of dfMedPatOrphans is 11 records difference, so quick analysis to see why
# dfMedLeftJoin = 4211, dfPatProcOrphans = 4208, diff 3
dfMedPatOrphansAnalysis = dfMedPatOrphans

In [47]:
dfMedPatOrphansAnalysis.rename(columns = {'PK':'_PK'}, inplace = True)

In [48]:
medPKi = dfMedLeftJoin.set_index('PK')
medPatPKi = dfMedPatOrphansAnalysis.set_index('_PK')

dfMedPatOrphansAnalysis = dfMedLeftJoin.merge(medPatPKi, 
                             how = 'left', 
                             left_on = ('PK'),
                             right_on = ('_PK'), 
                             indicator = 'i').query('i == "left_only"')

In [49]:
dfMedPatOrphansAnalysis.shape

(3, 14)

In [50]:
dfMedPatOrphansAnalysis

,Dim_Medication_PK_x,PatientId_x,Medication_Given_RXCUI_Code_x,Medication_Given_Description_x,FRDPersonnelID_x,PK,Dim_Medication_PK_y,PatientId_y,Medication_Given_RXCUI_Code_y,Medication_Given_Description_y,FRDPersonnelID_y,FRDPersonnelGender,FRDPersonnelStartDate,i
375,83144,531908,7213.0,Ipratropium Bromide(Atrovent),6AD7C99E-9E01-E211-B5F5-78E7D18CFD3C,531908_6AD7C99E-9E01-E211-B5F5-78E7D18CFD3C,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
376,83145,531908,435.0,Albuterol (Ventolin),6AD7C99E-9E01-E211-B5F5-78E7D18CFD3C,531908_6AD7C99E-9E01-E211-B5F5-78E7D18CFD3C,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
377,83146,531908,6902.0,Methylprednisolone (Solu-Medrol),6AD7C99E-9E01-E211-B5F5-78E7D18CFD3C,531908_6AD7C99E-9E01-E211-B5F5-78E7D18CFD3C,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only


In [51]:
# These 8 Provider IDs are not found at all in the Patients dataframe (or the Provider lookup dataframe)
dfMedPatOrphansAnalysis.groupby(['FRDPersonnelID_x']).size().reset_index(name='count')

,FRDPersonnelID_x,count
0,6AD7C99E-9E01-E211-B5F5-78E7D18CFD3C,3


# Join dfMedPatOrphans dataframe back with dfPatients to bring back in
```
PatientOutcome
PatientGender
```

In [52]:
# Reminder of dfPatients schema
dfPatients.columns

Index(['PatientId', 'FRDPersonnelID', 'Shift', 'UnitId', 'FireStation',
       'Battalion', 'PatientOutcome', 'PatientGender', 'CrewMemberRoles',
       'DispatchTime', 'FRDPersonnelGender', 'FRDPersonnelStartDate'],
      dtype='object')

In [53]:
# Reminder of dfMedPatOrphans schema
dfMedPatOrphans.columns

Index(['Dim_Medication_PK', 'PatientId', 'Medication_Given_RXCUI_Code',
       'Medication_Given_Description', 'FRDPersonnelID', '_PK',
       'FRDPersonnelGender', 'FRDPersonnelStartDate'],
      dtype='object')

In [54]:
luPatients = dfPatients.groupby(['PatientId', 'PatientOutcome', 'PatientGender','DispatchTime']).size().reset_index(name='count')
luPatients.shape

(189513, 5)

In [55]:
luPatients.dtypes

PatientId                  int64
PatientOutcome            object
PatientGender             object
DispatchTime      datetime64[ns]
count                      int64
dtype: object

In [56]:
luPatients.head()

,PatientId,PatientOutcome,PatientGender,DispatchTime,count
0,479851,Patient Refusal (AMA),Male,2018-01-01 00:20:49,2
1,479853,Treated & Transported,Male,2018-01-01 00:04:06,2
2,479862,Treated & Transported,Female,2018-01-01 00:44:31,4
3,479876,Treated & Transported,Male,2018-01-01 00:42:53,3
4,479879,Treated & Transported,Female,2018-01-01 01:17:54,2


In [57]:
# Drop count column, not needed
luPatients = luPatients.drop(['count'],axis=1)

In [58]:
dfMedPatSub = dfMedPatOrphans.merge(luPatients,on=('PatientId'))
dfMedPatSub.shape

(4182, 11)

In [59]:
dfMedPatSub.head()

,Dim_Medication_PK,PatientId,Medication_Given_RXCUI_Code,Medication_Given_Description,FRDPersonnelID,_PK,FRDPersonnelGender,FRDPersonnelStartDate,PatientOutcome,PatientGender,DispatchTime
0,65750,479920,237648.0,Dextrose 10 % (D10),C387923A-F613-E411-A585-F305C4522FCB,479920_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24,Treated & Transported,Female,2018-01-01 01:39:54
1,171166,607195,4337.0,Fentanyl Citrate (Sublimaze),C387923A-F613-E411-A585-F305C4522FCB,607195_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24,Treated & Transported,Female,2018-05-24 17:54:06
2,330909,625060,296.0,Adenosine (Adenocard),C387923A-F613-E411-A585-F305C4522FCB,625060_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24,Treated & Transported,Male,2018-06-13 08:18:44
3,330910,625060,296.0,Adenosine (Adenocard),C387923A-F613-E411-A585-F305C4522FCB,625060_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24,Treated & Transported,Male,2018-06-13 08:18:44
4,347351,674487,125464.0,Normal saline,C387923A-F613-E411-A585-F305C4522FCB,674487_C387923A-F613-E411-A585-F305C4522FCB,Male,2014-03-24,Treated & Transported,Male,2018-08-06 16:45:16


In [60]:
dfMedPatSub.columns

Index(['Dim_Medication_PK', 'PatientId', 'Medication_Given_RXCUI_Code',
       'Medication_Given_Description', 'FRDPersonnelID', '_PK',
       'FRDPersonnelGender', 'FRDPersonnelStartDate', 'PatientOutcome',
       'PatientGender', 'DispatchTime'],
      dtype='object')

In [61]:
(" \nCount total NaN at each column in a Medications dataFrame : \n\n", dfMedPatSub.isnull().sum())

(' \nCount total NaN at each column in a Medications dataFrame : \n\n',
 Dim_Medication_PK               0
 PatientId                       0
 Medication_Given_RXCUI_Code     0
 Medication_Given_Description    0
 FRDPersonnelID                  0
 _PK                             0
 FRDPersonnelGender              0
 FRDPersonnelStartDate           0
 PatientOutcome                  0
 PatientGender                   0
 DispatchTime                    0
 dtype: int64)

In [62]:
(" \nCount total NaN at each column in a Medications dataFrame : \n\n", dfMedPat.isnull().sum())

(' \nCount total NaN at each column in a Medications dataFrame : \n\n',
 PatientId                        0
 FRDPersonnelID                   0
 PatientOutcome                   0
 PatientGender                   32
 DispatchTime                     0
 FRDPersonnelGender               0
 FRDPersonnelStartDate            0
 Dim_Medication_PK                0
 Medication_Given_RXCUI_Code      6
 Medication_Given_Description     6
 _PK                              0
 dtype: int64)

# Append dfMedPatSub back to dfMedPat and do not reindex

In [63]:
dfMedPatAppend = dfMedPat.append(dfMedPatSub, ignore_index=True)
dfMedPatAppend.shape

(62881, 11)

In [64]:
(" \nCount total NaN at each column in a Medications dataFrame : \n\n", dfMedPatAppend.isnull().sum())

(' \nCount total NaN at each column in a Medications dataFrame : \n\n',
 PatientId                        0
 FRDPersonnelID                   0
 PatientOutcome                   0
 PatientGender                   32
 DispatchTime                     0
 FRDPersonnelGender               0
 FRDPersonnelStartDate            0
 Dim_Medication_PK                0
 Medication_Given_RXCUI_Code      6
 Medication_Given_Description     6
 _PK                              0
 dtype: int64)

In [65]:
print_categories(dfMedPatAppend)

PatientId :  Numerical
FRDPersonnelID :  Categorical
PatientOutcome :  Categorical
PatientGender :  Categorical
DispatchTime :  Date
FRDPersonnelGender :  Categorical
FRDPersonnelStartDate :  Date
Dim_Medication_PK :  Numerical
Medication_Given_RXCUI_Code :  Numerical
Medication_Given_Description :  Categorical
_PK :  Categorical


In [66]:
dfMedPatAppend['PatientOutcome'].value_counts()

Treated & Transported                        59537
Patient Dead at Scene (EMS CPR Attempted)     1855
Patient Refusal  (AMA)                        1343
Treated, Transferred Care                      112
EMS Assist (Other Agency)                       32
Patient Dead at Scene (No EMS CPR)               2
Name: PatientOutcome, dtype: int64

In [67]:
dfMedPatAppend['PatientGender'].value_counts()

Male                             32536
Female                           30289
Unknown (Unable to Determine)       24
Name: PatientGender, dtype: int64

In [68]:
dfMedPatAppend['FRDPersonnelGender'].value_counts()

Male      52356
Female    10525
Name: FRDPersonnelGender, dtype: int64

In [69]:
dfMedPatAppend['Medication_Given_Description'].value_counts()

Oxygen                                  10707
Normal saline                            7552
Ondansetron (Zofran)                     6855
Fentanyl Citrate (Sublimaze)             6577
Epinephrine 0.1 MG/ML (1:10,000)         6189
Acetylsalicylic Acid (Aspirin, ASA)      4349
Albuterol (Ventolin)                     3837
Ipratropium Bromide(Atrovent)            2817
Nitroglycerin                            2721
Naloxone (Narcan)                        2056
Midazolam (Versed)                       1395
Dextrose 10 % (D10)                      1187
Glucose (Oral)                            999
Methylprednisolone (Solu-Medrol)          981
Diphenhydramine (Benadryl)                915
Dextrose 50 % (D50)                       866
Ketamine (Ketalar)                        642
Epinephrine 1 MG/ML (1:1,000)             565
Adenosine (Adenocard)                     393
Atropine (Atropine Sulfate)               313
Amiodarone (Cordarone)                    294
Glucagon (Glucagen)               

# Add TenureMonths, which is the count of months working at the time of the dispatch call

In [70]:
# Add TenureMonths, which is the count of months working at the time of the dispatch call
dfMedPatAppend['TenureMonths'] = ((dfMedPatAppend.loc[:, 'DispatchTime'].dt.date - \
                                     dfMedPatAppend.loc[:, 'FRDPersonnelStartDate'].dt.date) / \
                                    np.timedelta64(1, 'M')).astype(int)
dfMedPatAppend.shape
# error is just a warning, need to figure out even though using loc

(62881, 12)

In [71]:
dfMedPatAppend['TenureMonths'].value_counts()

55     658
51     656
49     650
59     629
56     626
      ... 
440      1
420      1
419      1
414      1
430      1
Name: TenureMonths, Length: 420, dtype: int64

In [72]:
# First just going to to add a factorized column starting with 1 as opposed to the normal python 0
dfMedPatAppend['PatientOutcomeCode'] = pd.factorize(dfMedPatAppend['PatientOutcome'])[0] + 1

In [73]:
dfMedPatAppend['PatientOutcomeCode'].value_counts()

1    59537
4     1855
3     1343
2      112
5       32
6        2
Name: PatientOutcomeCode, dtype: int64

In [74]:
dfMedPatAppend['PatientGenderCode'] = pd.factorize(dfMedPatAppend['PatientGender'])[0] + 1

In [75]:
dfMedPatAppend['PatientGenderCode'].value_counts()

2    32536
1    30289
0       32
3       24
Name: PatientGenderCode, dtype: int64

In [76]:
# Adding "One Hot Encoding" method using get_dummies to create a row for each patient gender
dumDumPatient = pd.get_dummies(dfMedPatAppend['PatientGender'], prefix='PatientGender',)

In [77]:
dumDumPatient.head()

,PatientGender_Female,PatientGender_Male,PatientGender_Unknown (Unable to Determine)
0,1,0,0
1,0,1,0
2,0,1,0
3,0,1,0
4,1,0,0


In [78]:
dumDumPatient.value_counts()

PatientGender_Female  PatientGender_Male  PatientGender_Unknown (Unable to Determine)
0                     1                   0                                              32536
1                     0                   0                                              30289
0                     0                   0                                                 32
                                          1                                                 24
dtype: int64

In [79]:
dfMedPatAppend = pd.concat([dfMedPatAppend, dumDumPatient], axis=1)

In [80]:
dfMedPatAppend.head()

,PatientId,FRDPersonnelID,PatientOutcome,PatientGender,DispatchTime,FRDPersonnelGender,FRDPersonnelStartDate,Dim_Medication_PK,Medication_Given_RXCUI_Code,Medication_Given_Description,_PK,TenureMonths,PatientOutcomeCode,PatientGenderCode,PatientGender_Female,PatientGender_Male,PatientGender_Unknown (Unable to Determine)
0,479862,1D18E8FC-EE92-E211-A596-78E7D18C3D20,Treated & Transported,Female,2018-01-01 00:44:31,Female,2012-09-24,65740,7806.0,Oxygen,479862_1D18E8FC-EE92-E211-A596-78E7D18C3D20,63,1,1,1,0,0
1,479876,35CA45A3-9C25-4E4D-957F-2EA9E40B5974,Treated & Transported,Male,2018-01-01 00:42:53,Male,2016-02-08,65744,26225.0,Ondansetron (Zofran),479876_35CA45A3-9C25-4E4D-957F-2EA9E40B5974,22,1,2,0,1,0
2,479916,F73B4EF0-B70C-4960-9B92-8BF196A30B57,Treated & Transported,Male,2018-01-01 01:48:31,Male,2016-02-08,65749,125464.0,Normal saline,479916_F73B4EF0-B70C-4960-9B92-8BF196A30B57,22,1,2,0,1,0
3,480011,08CC5399-F492-E211-A596-78E7D18C3D20,Treated & Transported,Male,2018-01-01 03:02:53,Male,2012-09-24,65758,125464.0,Normal saline,480011_08CC5399-F492-E211-A596-78E7D18C3D20,63,1,2,0,1,0
4,480022,E10FD6B2-19A1-468A-825B-7EE0061DBB01,Treated & Transported,Female,2018-01-01 03:04:20,Male,2016-09-19,65759,26225.0,Ondansetron (Zofran),480022_E10FD6B2-19A1-468A-825B-7EE0061DBB01,15,1,1,1,0,0


In [81]:
dfMedPatAppend['ProviderGenderCode'] = pd.factorize(dfMedPatAppend['FRDPersonnelGender'])[0] + 1

In [82]:
dfMedPatAppend['ProviderGenderCode'].value_counts()

2    52356
1    10525
Name: ProviderGenderCode, dtype: int64

In [83]:
print_categories(dfMedPatAppend)

PatientId :  Numerical
FRDPersonnelID :  Categorical
PatientOutcome :  Categorical
PatientGender :  Categorical
DispatchTime :  Date
FRDPersonnelGender :  Categorical
FRDPersonnelStartDate :  Date
Dim_Medication_PK :  Numerical
Medication_Given_RXCUI_Code :  Numerical
Medication_Given_Description :  Categorical
_PK :  Categorical
TenureMonths :  Numerical
PatientOutcomeCode :  Numerical
PatientGenderCode :  Numerical
PatientGender_Female :  Numerical
PatientGender_Male :  Numerical
PatientGender_Unknown (Unable to Determine) :  Numerical
ProviderGenderCode :  Numerical


In [84]:
dfMedPatAppend.to_csv(r'./data/MedicationsPatients-20210225-ems-raw-v04.csv', index = True)